In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install -q py-climate-health-toolbox
import pandas as pd
import numpy as np
import climahe.climatex as tex

**Leitura dos dados**

In [3]:
df2015 = pd.read_excel('/content/drive/Shared drives/Clima&Saúde/Pesquisadores/Gitti_JoãoVictor/Dados - Poluição/Dados tratados/Taquaral_MP10/Taquaral_MP10_2015.xlsx')
df2016 = pd.read_excel('/content/drive/Shared drives/Clima&Saúde/Pesquisadores/Gitti_JoãoVictor/Dados - Poluição/Dados tratados/Taquaral_MP10/Taquaral_MP10_2016.xlsx')
df2017 = pd.read_excel('/content/drive/Shared drives/Clima&Saúde/Pesquisadores/Gitti_JoãoVictor/Dados - Poluição/Dados tratados/Taquaral_MP10/Taquaral_MP10_2017.xlsx')
df2018 = pd.read_excel('/content/drive/Shared drives/Clima&Saúde/Pesquisadores/Gitti_JoãoVictor/Dados - Poluição/Dados tratados/Taquaral_MP10/Taquaral_MP10_2018.xlsx')
df2019 = pd.read_excel('/content/drive/Shared drives/Clima&Saúde/Pesquisadores/Gitti_JoãoVictor/Dados - Poluição/Dados tratados/Taquaral_MP10/Taquaral_MP10_2019.xlsx')
df2020 = pd.read_excel('/content/drive/Shared drives/Clima&Saúde/Pesquisadores/Gitti_JoãoVictor/Dados - Poluição/Dados tratados/Taquaral_MP10/Taquaral_MP10_2020.xlsx')

**Preparação dos dados**

In [4]:
df = pd.concat([df2015, df2016, df2017, df2018, df2019, df2020])
df['Data'] = pd.to_datetime(df['Data'], format='%d/%m/%Y')
df = df.dropna(subset = ['Média Horária'])
df = df.rename(columns={'Data': 'DATE'})

df_con_max = df.groupby('DATE').max()
df_con_min = df.groupby('DATE').min()

df_con_max = df_con_max.rename(columns={'Média Horária': 'CON_MAX'})
df_con_min = df_con_min.rename(columns={'Média Horária': 'CON_MIN'})

merge = pd.merge(df_con_max['CON_MAX'],df_con_min['CON_MIN'],on='DATE',how='left')
merge = merge.reset_index()
merge = tex.complete_df(merge)
merge = tex.date_toDay365(merge)
merge = tex.drop_leapday(merge)
df_final = merge
df_final['YEAR'] = df_final['DATE'].dt.year

**Dataset gerado para definir a normal de poluição**

In [5]:
normal_pol = df_final[(df_final['DATE']>='2016-01-01')&(df_final['DATE']<='2019-12-31')]
normal_pol=normal_pol.reset_index()
del normal_pol['index']
normal_pol['pct_column'] = np.nan

**Database gerado para análise**

In [6]:
database = df_final[(df_final['DATE']>='2015-01-01')&(df_final['DATE']<='2020-12-31')]

**Criação do DataFrame para definir o mínimo e máximo de poluição das ondas**

In [7]:
df_pct = pd.DataFrame() 
df_pct['DAY365']=range(1,366) 
df_pct=df_pct.set_index(['DAY365']) 

**Inicialmente, utilizou-se o percentil 90 de dia para dia**

df_pct['MAX90pct'] = df_pct.index.map(lambda day365_index: 
                                      tex.get_percentile(normal_pol,day365_index,'CON_MAX',0.9,15))
df_pct['MIN90pct'] = df_pct.index.map(lambda day365_index: 
                                      tex.get_percentile(normal_pol,day365_index,'CON_MIN',0.9,15))

**Em seguida, considerou-se o percentil 90 para todo o Dataset**

df_pct['MAX90pct'] = normal_pol['CON_MAX'].quantile(0.9)
df_pct['MIN90pct'] = normal_pol['CON_MIN'].quantile(0.9)

**Finalmente, o DataFrame foi contruído com base no nível crítico de poluição**

In [8]:
df_pct['MAX90pct'] = 100
df_pct['MIN90pct'] = 0

In [9]:
df_aux=pd.DataFrame()
database.set_index(['DAY365'],inplace=True) 

for year in database.DATE.dt.year.unique(): 
    df_year = database[database.DATE.dt.year == year]
    df_year.loc[:,'above_pct'] = 0
    mask_pw = (df_year['CON_MAX'] >= df_pct['MAX90pct']) & (df_year['CON_MIN'] >= df_pct['MIN90pct'])
    df_year.loc[mask_pw,'above_pct'] = 1 
    if df_aux is None: 
        df_aux = df_year
    else:
        df_aux=df_aux.append(df_year) 
    
df_aux=df_aux.reset_index()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

In [10]:
df_wave = df_aux

In [11]:
check_pattern = df_wave.rolling(3)['above_pct'].apply(lambda x: all(np.equal(x, [1,1,1])),raw=True) 
check_pattern = check_pattern.fillna(False).astype(bool)
pattern_idx = np.where(check_pattern)[0] 
    
subset = [range(idx-3+1, idx+1) for idx in pattern_idx] 
     
idx_list = [item for sublist in subset for item in sublist] 
    
idx_list=list(set(idx_list))  
    
df_wave['PW']=0 
df_wave['PW'].iloc[idx_list]=1 

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


**Dias de onda de poluição**

In [12]:
df_wave[df_wave['PW']==1]

,DAY365,DATE,CON_MAX,CON_MIN,YEAR,above_pct,PW
2095,271,2020-09-28,132.0,3.0,2020,1,1
2096,272,2020-09-29,106.0,8.0,2020,1,1
2097,273,2020-09-30,107.0,0.0,2020,1,1
2098,274,2020-10-01,111.0,0.0,2020,1,1
2099,275,2020-10-02,115.0,0.0,2020,1,1
2102,278,2020-10-05,116.0,2.0,2020,1,1
2103,279,2020-10-06,128.0,0.0,2020,1,1
2104,280,2020-10-07,149.0,23.0,2020,1,1


**Níveis de poluição:**

**Boa: 0 ~ 50 (µg/m3)**

**Moderada: 50 ~ 100 (µg/m3)**

**Ruim: 100 ~ 150 (µg/m3)**

**Muito ruim: 150 ~ 250 (µg/m3)**

**Péssima: > 250 (µg/m3)**